In [1]:
#P Value Analysis
import csv
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Team-Folder/Cleaned_RandomSamples/')


In [2]:
Jan = pd.read_csv("RideShare_Data_201801.csv", parse_dates=[0], low_memory=False)
Feb = pd.read_csv("RideShare_Data_201802.csv", parse_dates=[0], low_memory=False)
Mar = pd.read_csv("RideShare_Data_201803.csv", parse_dates=[0], low_memory=False)
Apr = pd.read_csv("RideShare_Data_201804.csv", parse_dates=[0], low_memory=False)
May = pd.read_csv("RideShare_Data_201805.csv", parse_dates=[0], low_memory=False)
Jun = pd.read_csv("RideShare_Data_201806.csv", parse_dates=[0], low_memory=False)
Jul = pd.read_csv("RideShare_Data_201807.csv", parse_dates=[0], low_memory=False)
Aug = pd.read_csv("RideShare_Data_201808.csv", parse_dates=[0], low_memory=False)
Sep = pd.read_csv("RideShare_Data_201809.csv", parse_dates=[0], low_memory=False)
Oct = pd.read_csv("RideShare_Data_201810.csv", parse_dates=[0], low_memory=False)
Nov = pd.read_csv("RideShare_Data_201811.csv", parse_dates=[0], low_memory=False)
Dec = pd.read_csv("RideShare_Data_201812.csv", parse_dates=[0], low_memory=False)
Yearly_Data = pd.concat([Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec])

Yearly_Data['Pickup_DateTime'] = Yearly_Data['Pickup_DateTime'].astype('datetime64[ns]') 
Yearly_Data['Month'] = Yearly_Data['Pickup_DateTime'].dt.month
Yearly_Data['Day_Of_Week'] = Yearly_Data['Pickup_DateTime'].dt.dayofweek
Yearly_Data['Time'] = [d.time() for d in Yearly_Data['Pickup_DateTime']]
Yearly_Data[['Hour','Minute','Second']] = Yearly_Data['Time'].astype(str).str.split(':', expand=True).astype(int)

Yearly_Data.head()

,#,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Pickup_City,Dropoff_City,Pickup_ZipCode,Dropoff_ZipCode,...,wind_speed,weather_id,weather_main,weather_description,Month,Day_Of_Week,Time,Hour,Minute,Second
0,0,2018-01-01 00:00:02,2018-01-01 00:34:21,148.0,48.0,0.0,"Lower East Side, Manhattan","Clinton East, Manhattan",10024,10024,...,4.0,800.0,Clear,sky is clear,1,0,00:00:02,0,0,2
1,1,2018-01-01 00:04:52,2018-01-01 00:19:04,65.0,61.0,1.0,"Downtown Brooklyn/MetroTech, Brooklyn","Crown Heights North, Brooklyn",11201,11216,...,4.0,800.0,Clear,sky is clear,1,0,00:04:52,0,4,52
2,2,2018-01-01 00:05:05,2018-01-01 00:18:08,114.0,148.0,0.0,"Greenwich Village South, Manhattan","Lower East Side, Manhattan",10024,10024,...,4.0,800.0,Clear,sky is clear,1,0,00:05:05,0,5,5
3,3,2018-01-01 00:05:41,2018-01-01 00:26:39,79.0,189.0,0.0,"East Village, Manhattan","Prospect Heights, Brooklyn",10024,11216,...,4.0,800.0,Clear,sky is clear,1,0,00:05:41,0,5,41
4,4,2018-01-01 00:06:22,2018-01-01 00:26:00,7.0,130.0,0.0,"Astoria, Queens","Jamaica, Queens",11366,11366,...,4.0,800.0,Clear,sky is clear,1,0,00:06:22,0,6,22


In [3]:
#Create the buckets for the time of day
#Morning is from sunrise to 11:59 AM. Sunrise typically occurs around 6 AM. 6 to 11
#Afternoon is from 12:01 PM to around 5:00 PM. 12 to 17
#Evening is from 5:01 PM to 8 PM, or around sunset. 18 to 20
#Night is from sunset to sunrise, so from 8:01 PM until 5:59 AM. 21 to 5

Yearly_Data=Yearly_Data.assign(day_bucket=pd.cut(Yearly_Data.Pickup_DateTime.dt.hour,
                            [-1,6,12,18,23],
                            labels=['Night','Morning','Afternoon','Evening'],
                            include_lowest=True))

#Yearly_Data['day_bucket'] = Yearly_Data['day_bucket'].astype(int) 

# Create bins in which to place values based upon TED Talk views
Yearly_Data.head()


,#,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Pickup_City,Dropoff_City,Pickup_ZipCode,Dropoff_ZipCode,...,weather_id,weather_main,weather_description,Month,Day_Of_Week,Time,Hour,Minute,Second,day_bucket
0,0,2018-01-01 00:00:02,2018-01-01 00:34:21,148.0,48.0,0.0,"Lower East Side, Manhattan","Clinton East, Manhattan",10024,10024,...,800.0,Clear,sky is clear,1,0,00:00:02,0,0,2,Night
1,1,2018-01-01 00:04:52,2018-01-01 00:19:04,65.0,61.0,1.0,"Downtown Brooklyn/MetroTech, Brooklyn","Crown Heights North, Brooklyn",11201,11216,...,800.0,Clear,sky is clear,1,0,00:04:52,0,4,52,Night
2,2,2018-01-01 00:05:05,2018-01-01 00:18:08,114.0,148.0,0.0,"Greenwich Village South, Manhattan","Lower East Side, Manhattan",10024,10024,...,800.0,Clear,sky is clear,1,0,00:05:05,0,5,5,Night
3,3,2018-01-01 00:05:41,2018-01-01 00:26:39,79.0,189.0,0.0,"East Village, Manhattan","Prospect Heights, Brooklyn",10024,11216,...,800.0,Clear,sky is clear,1,0,00:05:41,0,5,41,Night
4,4,2018-01-01 00:06:22,2018-01-01 00:26:00,7.0,130.0,0.0,"Astoria, Queens","Jamaica, Queens",11366,11366,...,800.0,Clear,sky is clear,1,0,00:06:22,0,6,22,Night


In [4]:
p_value_data = Yearly_Data[["day_bucket","Trip_Duration"]]
p_value_data.head()


,day_bucket,Trip_Duration
0,Night,34.0
1,Night,14.0
2,Night,13.0
3,Night,21.0
4,Night,20.0


In [5]:
p_value_data = p_value_data.groupby('day_bucket')


In [6]:
p_value_data = p_value_data.Trip_Duration.mean()
p_value_data = p_value_data.reset_index()
p_value_data

,day_bucket,Trip_Duration
0,Night,17.870771
1,Morning,20.543959
2,Afternoon,22.267852
3,Evening,18.442345


In [7]:
p_value_data['expected'] = ( p_value_data.iloc[0]['Trip_Duration'] + p_value_data.iloc[1]['Trip_Duration'] + p_value_data.iloc[2]['Trip_Duration'] + p_value_data.iloc[3]['Trip_Duration'] ) / 4
p_value_data
                            

,day_bucket,Trip_Duration,expected
0,Night,17.870771,19.781232
1,Morning,20.543959,19.781232
2,Afternoon,22.267852,19.781232
3,Evening,18.442345,19.781232


In [8]:
critical_value = stats.chi2.ppf(q = 0.95, df = 2)
critical_value


5.991464547107979

In [9]:
#Time of Day and Trip Duration Chi Squared. Not significant.
stats.chisquare(p_value_data['Trip_Duration'], p_value_data['expected'])


Power_divergenceResult(statistic=0.6171258141255158, pvalue=0.8925008608673507)

In [10]:
p_value_data_c = Yearly_Data[["day_bucket","#"]]
p_value_data_c.head()


,day_bucket,#
0,Night,0
1,Night,1
2,Night,2
3,Night,3
4,Night,4


In [11]:
p_value_data_c = p_value_data_c.groupby('day_bucket')


In [12]:
p_value_data_c = p_value_data_c['#'].count()
p_value_data_c = p_value_data_c.reset_index()
p_value_data_c


,day_bucket,#
0,Night,158277
1,Morning,267796
2,Afternoon,323877
3,Evening,295163


In [13]:
p_value_data_c['expected'] = ( p_value_data_c.iloc[0]['#'] + p_value_data_c.iloc[1]['#'] + p_value_data_c.iloc[2]['#'] + p_value_data_c.iloc[3]['#'] ) / 4
p_value_data_c
               

,day_bucket,#,expected
0,Night,158277,261278.25
1,Morning,267796,261278.25
2,Afternoon,323877,261278.25
3,Evening,295163,261278.25


In [14]:
critical_value = stats.chi2.ppf(q = 0.95, df = 2)
critical_value


5.991464547107979

In [15]:
#Time of Day and Trip Count Chi Squared. Significant.
stats.chisquare(p_value_data_c['#'], p_value_data_c['expected'])


Power_divergenceResult(statistic=60160.07207163245, pvalue=0.0)

In [16]:
p_value_data_h = Yearly_Data[["humidity","#"]]
p_value_data_h.head()


,humidity,#
0,43.0,0
1,43.0,1
2,43.0,2
3,43.0,3
4,43.0,4


In [17]:
p_value_data_h = p_value_data_h.groupby('humidity')


In [18]:
p_value_data_h = p_value_data_h['#'].count()
p_value_data_h = p_value_data_h.reset_index()
p_value_data_h.head()


,humidity,#
0,12.0,414
1,13.0,145
2,14.0,776
3,15.0,510
4,16.0,488


In [19]:
p_value_data_h['expected'] = ( p_value_data_h['#'].sum() / p_value_data_h['#'].count() )
p_value_data_h.head()


,humidity,#,expected
0,12.0,414,11742.842697
1,13.0,145,11742.842697
2,14.0,776,11742.842697
3,15.0,510,11742.842697
4,16.0,488,11742.842697


In [20]:
critical_value = stats.chi2.ppf(q = 0.95, df = 2)
critical_value


5.991464547107979

In [21]:
#Humidity and Trip Count Chi Squared. Significant.
stats.chisquare(p_value_data_h['#'], p_value_data_h['expected'])


Power_divergenceResult(statistic=671281.9201827936, pvalue=0.0)

In [22]:
p_value_data_t = Yearly_Data[["Temp","#"]]
p_value_data_t.head()


,Temp,#
0,11,0
1,11,1
2,11,2
3,11,3
4,11,4


In [23]:
p_value_data_t = p_value_data_t.groupby('Temp')


In [24]:
p_value_data_t = p_value_data_t['#'].count()
p_value_data_t = p_value_data_t.reset_index()
p_value_data_t.head()


,Temp,#
0,2,278
1,3,128
2,4,227
3,5,124
4,6,555


In [25]:
p_value_data_t['expected'] = ( p_value_data_t['#'].sum() / p_value_data_t['#'].count() )
p_value_data_t.head()


,Temp,#,expected
0,2,278,11237.774194
1,3,128,11237.774194
2,4,227,11237.774194
3,5,124,11237.774194
4,6,555,11237.774194


In [26]:
critical_value = stats.chi2.ppf(q = 0.95, df = 2)
critical_value


5.991464547107979

In [27]:
#There is significance between temp and trip count
stats.chisquare(p_value_data_t['#'], p_value_data_t['expected'])


Power_divergenceResult(statistic=484648.6436241823, pvalue=0.0)

In [34]:
p_value_data_h_td = Yearly_Data[["humidity","Trip_Duration"]]
p_value_data_h_td.head()


,humidity,Trip_Duration
0,43.0,34.0
1,43.0,14.0
2,43.0,13.0
3,43.0,21.0
4,43.0,20.0


In [35]:
p_value_data_h_td = p_value_data_h_td.groupby('humidity')


In [36]:
p_value_data_h_td = p_value_data_h_td['Trip_Duration'].mean()
p_value_data_h_td = p_value_data_h_td.reset_index()
p_value_data_h_td.head()


,humidity,Trip_Duration
0,12.0,18.157005
1,13.0,21.379310
2,14.0,19.012887
3,15.0,20.425490
4,16.0,21.366803


In [37]:
p_value_data_h_td['expected'] = ( p_value_data_h_td['Trip_Duration'].sum() / p_value_data_h_td['Trip_Duration'].count() )
p_value_data_h_td.head()


,humidity,Trip_Duration,expected
0,12.0,18.157005,20.050299
1,13.0,21.379310,20.050299
2,14.0,19.012887,20.050299
3,15.0,20.425490,20.050299
4,16.0,21.366803,20.050299


In [38]:
critical_value = stats.chi2.ppf(q = 0.95, df = 2)
critical_value


5.991464547107979

In [39]:
#Humidity and Trip Duration is not showing significant trend
stats.chisquare(p_value_data_h_td['Trip_Duration'], p_value_data_t_td['expected'])


Power_divergenceResult(statistic=1.8549465705190662, pvalue=1.0)

In [58]:
p_value_data_t_td = Yearly_Data[["Temp","Trip_Duration"]]
p_value_data_t_td.head()


,Temp,Trip_Duration
0,11,34.0
1,11,14.0
2,11,13.0
3,11,21.0
4,11,20.0


In [59]:
p_value_data_t_td = p_value_data_t_td.groupby('Temp')


In [60]:
p_value_data_t_td = p_value_data_t_td['Trip_Duration'].mean()
p_value_data_t_td = p_value_data_t_td.reset_index()
p_value_data_t_td.head()


,Temp,Trip_Duration
0,2,16.521583
1,3,15.960938
2,4,14.920705
3,5,15.451613
4,6,17.981982


In [61]:
p_value_data_t_td['expected'] = ( p_value_data_t_td['Trip_Duration'].sum() / p_value_data_t_td['Trip_Duration'].count() )
p_value_data_t_td.head()


,Temp,Trip_Duration,expected
0,2,16.521583,19.512999
1,3,15.960938,19.512999
2,4,14.920705,19.512999
3,5,15.451613,19.512999
4,6,17.981982,19.512999


In [63]:
#Temp and Trip Duration is not showing significant trend
stats.chisquare(p_value_data_t_td['Trip_Duration'], p_value_data_t_td['expected'])


Power_divergenceResult(statistic=7.560755713356393, pvalue=1.0)

In [46]:
p_value_data_sr_c = Yearly_Data[["SR_Flag","#"]]
p_value_data_sr_c.head()


,SR_Flag,#
0,0.0,0
1,1.0,1
2,0.0,2
3,0.0,3
4,0.0,4


In [48]:
p_value_data_sr_c = p_value_data_sr_c.groupby('SR_Flag')


In [49]:
p_value_data_sr_c = p_value_data_sr_c['#'].count()
p_value_data_sr_c = p_value_data_sr_c.reset_index()
p_value_data_sr_c.head()


,SR_Flag,#
0,0.0,743463
1,1.0,301650


In [50]:
p_value_data_sr_c['expected'] = ( p_value_data_sr_c['#'].sum() / p_value_data_sr_c['#'].count() )
p_value_data_sr_c.head()


,SR_Flag,#,expected
0,0.0,743463,522556.5
1,1.0,301650,522556.5


In [51]:
#SR Flag and Trip Count is showing significant trend
stats.chisquare(p_value_data_sr_c['#'], p_value_data_sr_c['expected'])


Power_divergenceResult(statistic=186772.8436724067, pvalue=0.0)

In [52]:
p_value_data_sr_td = Yearly_Data[["SR_Flag","Trip_Duration"]]
p_value_data_sr_td.head()


,SR_Flag,Trip_Duration
0,0.0,34.0
1,1.0,14.0
2,0.0,13.0
3,0.0,21.0
4,0.0,20.0


In [53]:
p_value_data_sr_td = p_value_data_sr_td.groupby('SR_Flag')


In [54]:
p_value_data_sr_td = p_value_data_sr_td['Trip_Duration'].mean()
p_value_data_sr_td = p_value_data_sr_td.reset_index()
p_value_data_sr_td.head()


,SR_Flag,Trip_Duration
0,0.0,19.323865
1,1.0,21.942934


In [55]:
p_value_data_sr_td['expected'] = ( p_value_data_sr_td['Trip_Duration'].sum() / p_value_data_sr_td['Trip_Duration'].count() )
p_value_data_sr_td.head()


,SR_Flag,Trip_Duration,expected
0,0.0,19.323865,20.6334
1,1.0,21.942934,20.6334


In [56]:
#SR Flag and Trip Duration is not showing significant trend
stats.chisquare(p_value_data_sr_td['Trip_Duration'], p_value_data_sr_td['expected'])


Power_divergenceResult(statistic=0.16622367902783775, pvalue=0.6834899852536357)